# Statistik „Mathe für Nicht-Freaks“ 2016

In [1]:
# Helper functions for Mediawiki API

import requests

from datetime import datetime

def query_json(path, json):
    result = json
    
    for key in path:
        if callable(key):
            result = key(result)
        else:
            result = result[key]
            
    return result

select_singleton_dict = lambda x: list(x.values())[0]

def api(params):
    params["format"] = "json"
    
    return requests.get("https://de.wikibooks.org/w/api.php", params).json()

def query(params, path):
    params["action"] = "query"
    
    json = api(params)
    result = query_json(["query"] + path, json)
    
    if "query-continue" in json:
        params.update(query_json(["query_continue", select_singleton_list], json))
        result += query(params, json)
    
    return result

In [2]:
# Get list of all pages

def list_pages():
    return query({
            "list": "allpages",
            "apprefix": "Mathe für Nicht-Freaks",
            "apfilterredir": "nonredirects",
            "aplimit": 500
        }, ["allpages"])

pages = list_pages()

In [3]:
# Query all revisions and views

def add_revisions(page):
    revs = query({
            "prop": "revisions",
            "titles": page["title"],
            "rvlimit": 500,
            "rvprop": "ids|timestamp|user|userid|size"
        }, ["pages", select_singleton_dict, "revisions"])
    
    old_size = 0
    
    for rev in reversed(revs):
        rev["diff"] = rev["size"] - old_size
        old_size = rev["size"]
        rev["timestamp"] = datetime.strptime(rev["timestamp"], "%Y-%m-%dT%H:%M:%SZ")
    
    page["revisions"] = revs
    
    return revs

revs = []

for page in pages:
    revs += add_revisions(page)
    
def add_views2016(page):
    url = ("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/" +
           "de.wikibooks.org/all-access/all-agents/%s/daily/20160101/20161231")
    
    views = requests.get(url % page["title"].replace("/", "%2F").replace("?", "%3F")).json()["items"]
    
    page["views"] = sum(map(lambda x: int(x["views"]), views))

for page in pages:
    add_views2016(page)

# Anzahl Bearbeitungen 2016

In [4]:
nrevs2016 = len(list(filter(lambda x: x["timestamp"].year == 2016, revs)))
nrevs2015 = len(list(filter(lambda x: x["timestamp"].year == 2015, revs)))

print("Anzahl Bearbeitungen 2016: %s" % nrevs2016)
print("Anzahl Bearbeitungen 2015: %s" % nrevs2015)
print("Diff: %s" % (nrevs2016-nrevs2015))

Anzahl Bearbeitungen 2016: 4731
Anzahl Bearbeitungen 2015: 1807
Diff: 2924


# Anzahl neuer Artikel 2016

In [5]:
newpages = len(list(filter(lambda x: x["revisions"][-1]["timestamp"].year==2016, pages)))

print("Neue Artikel 2016: %s" % newpages)
print("Zuwachs: %s" % (newpages/(len(pages)-newpages)))
print("Gesamtanzahl an Artikel: %s" % len(pages))

Neue Artikel 2016: 91
Zuwachs: 0.4945652173913043
Gesamtanzahl an Artikel: 275


# Statistik zu Autoren / Autorinnen

In [6]:
from itertools import groupby, islice

revs2016 = filter(lambda x: x["timestamp"].year==2016, revs)
noauthors = len(set(map(lambda x: x["user"], revs)))
keyfunc = lambda x: x["user"]
authors2016 = list(map(lambda x: ( x[0], len(list(x[1])) ), groupby(sorted(revs2016, key=keyfunc), keyfunc)))

print("Gesamtzahl Autoren / Autorinnen: %s" % noauthors)
print("Autoren/Autorinnen 2016: %s" % len(authors2016))
print()
print("Liste der ersten 30 Autoren / Autorinnen nach Bearbeitungen:")
print()

for author, no in islice(sorted(authors2016, key=lambda x: x[1], reverse=True), 0, 30):
    print("%4d – %s" % (no, author))

Gesamtzahl Autoren / Autorinnen: 572
Autoren/Autorinnen 2016: 178

Liste der ersten 30 Autoren / Autorinnen nach Bearbeitungen:

1380 – Stephan Kulla
1194 – Who2010
 804 – Matheoldie
 445 – Auswahlaxiom
  91 – DominikJW
  61 – Claudia4
  53 – 62.12.154.122
  46 – Christoph Kehle
  45 – Mrvnfrtz
  42 – 0-Brane
  42 – S jwiese
  35 – Stefanie Fenkl
  32 – Dirk Hünniger
  32 – GraffL
  31 – Mathpro01
  28 – Meitnerium266
  20 – Morpurgo10
  17 – Oztafankolibril
  14 – Mattlocke2.0
  14 – Peter Gröbner
  14 – Talonnn
  11 – $pecial
  10 – Fabiangabel
   9 – 188.194.65.153
   9 – Sven87a
   9 – Theresa Plomer
   7 – 188.193.138.47
   7 – 2003:86:2E26:93C:7DAE:E4B9:77AA:5873
   6 – 188.174.169.87
   6 – 91.36.202.45


# Seitenaufrufe 2016

In [7]:
views2016 = sum(map(lambda x: x["views"], pages))

print("Seitenaufrufe 2016: %s" % views2016)

Seitenaufrufe 2016: 1654402


# Seitenaufrufe nach Artikel

In [8]:
for page in sorted(pages, key = lambda x: x["views"], reverse = True):
    print("%5d – %s" % (page["views"], page["title"]))

76825 – Mathe für Nicht-Freaks: Vollständige Induktion
54091 – Mathe für Nicht-Freaks: Grenzwert: Konvergenz und Divergenz
53940 – Mathe für Nicht-Freaks
49492 – Mathe für Nicht-Freaks: Analysis 1
45523 – Mathe für Nicht-Freaks: Grundlagen der Mathematik
41940 – Mathe für Nicht-Freaks: Konvergenz und Divergenz beweisen
39780 – Mathe für Nicht-Freaks: Binomialkoeffizient
39527 – Mathe für Nicht-Freaks: Summe und Produkt
36337 – Mathe für Nicht-Freaks: Geometrische Reihe
33124 – Mathe für Nicht-Freaks: Abbildung, Funktion
32695 – Mathe für Nicht-Freaks: Geometrische Summenformel
32251 – Mathe für Nicht-Freaks: Vollständige Induktion: Beispiele
31598 – Mathe für Nicht-Freaks: Grenzwert: Beispiele
28770 – Mathe für Nicht-Freaks: Aussagen negieren
26620 – Mathe für Nicht-Freaks: Binomialkoeffizient: Rechenregeln
25826 – Mathe für Nicht-Freaks: Grenzwertsätze: Grenzwert von Folgen berechnen
25495 – Mathe für Nicht-Freaks: Äquivalenzrelation
25323 – Mathe für Nicht-Freaks: Explizite und rekur